In [ ]:
%load 1.1.train_model_weight.py
%run 1.1.train_model_weight.py

validation loss = 111.208841: 100%|██████████| 47/47 [00:03<00:00, 15.07it/s]



epoch 1, best valid loss = inf,count = 0


validation loss = 91.586264: 100%|██████████| 47/47 [00:03<00:00, 14.50it/s]



epoch 2, best valid loss = inf,count = 0


validation loss = 93.927444: 100%|██████████| 47/47 [00:03<00:00, 15.40it/s]


...

validation loss = 89.945499: 100%|██████████| 47/47 [00:02<00:00, 15.71it/s]



epoch 54, best valid loss = 4130.55457306,count = 11


validation loss = 89.496848: 100%|██████████| 47/47 [00:02<00:00, 15.68it/s]



epoch 55, best valid loss = 4130.55457306,count = 12


train loss = 89.395947:  73%|███████▎  | 241/329 [00:11<00:03, 23.47it/s]

In [ ]:
!/usr/bin/shutdown